Finding Stabilizer States using Gates' Symplectic Forms
===

This code finds all 60 possible combinations of the two qubit gate stabilizer gates, and uses them to find the corresponding stabilizer states. It does this using the following formula:

$$\rho_{\left|\psi\right\rangle} = \left|\psi\right\rangle \left\langle\psi\right| \propto \mathbf{I} \pm g_1 \pm g_2 \pm g_1 \cdot g_2$$
where:

* $\rho_{\left|\psi\right\rangle}$ is the density matrix of the quantum state $\left|\psi\right\rangle$, which in the two qubit case should be a four by four matrix
* $\mathbf{I}$ is the identity matrix, which in the two qubit case is $\mathbf{I}_4$
* $g_1$ and $g_2$ are stabilizer gates of the state $\left|\psi\right\rangle$
* the $g_1 \cdot g_2$ multiplication results in another stabilizer gate of the state $\left|\psi\right\rangle$.

In [ ]:
import numpy as np
from numpy.linalg import eig

In [ ]:
#this function converts a gate from Pauli form to symplectic
#it expects the gate in a string form
def symplectify(gate):
    gate = list(gate)
    
    minus=False
    imag =False
    
    if "-" in gate:               #checks if the gate has a minus phase
        index = gate.index("-")   #also removes the minus such that the symplectic form is the right shape
        gate.pop(index)
        minus=True
    if "i" in gate:               #checks if the gate has an imaginary phase
        index = gate.index("i")   #also removes the imaginary number such that the symplectic form is the right shape
        gate.pop(index)
        imag=True
    
    form = [np.zeros(len(gate)),np.zeros(len(gate)),np.array([0])]        #set up the shape of the symplectic form
    
    if minus == True:               #this governs the phase bit of the symplectic form
        form[2][0] += 1
    if imag == True:                #later on our gates should never have an imaginary phase, -
        print("Unwanted phase")     #- so if we have one we have an issue
    
    for S in gate:                  #here we run over every tensored gate
        if S == "X":                #if the gate is an "X", the symplectic form's first column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1

        if S == "Z":                #if the gate is a "Z", the symplectic form's second column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[1][index] += 1

        if S == "Y":                #if the gate is a "Y", the symplectic form's first and second columns are updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1
            form[1][index] += 1
            
    return form                     #returns the gate's symplectic form



#this function calculates the phase change produced when you multiply two gates together
def g_(a,b,c,d):
    phase=0
    for i in range(len(a)):
        phase += a[i]**2*b[i]**2 + c[i]**2*d[i]**2 - (a[i]+c[i])**2*(b[i]+d[i])**2 + 2*c[i]*b[i]
    return phase%4



#this function multiplies two gates together
#it expects the gates in the symplectic form
def symp_mul(gate1, gate2):
    new_gate = [np.zeros(len(gate1[0])),np.zeros(len(gate1[0])),np.array([0])]
    new_gate[0] = (gate1[0] + gate2[0])%2
    new_gate[1] = (gate1[1] + gate2[1])%2
    new_gate[2] = ((gate1[2]*2 + gate2[2]*2 + g_(gate1[0],gate1[1],gate2[0],gate2[1]))%4)/2
    
    return new_gate



#this function converts the gate from symplectic form to Pauli form
#it expects the gate to be in the symplectic form
def Paulify(symp_gate):
    Pgate = []
    for i in range(len(symp_gate[0])):
        if symp_gate[0][i] == 1 and symp_gate[1][i] == 1: #if theres a 1 in both q and p, the ith gate is Y
            Pgate.append("Y")
        elif symp_gate[0][i] == 1:                        #elif theres a 1 in q only, the ith gate is X
            Pgate.append("X")
        elif symp_gate[1][i] == 1:                        #elif theres a 1 in p only, the ith gate is Z
            Pgate.append("Z")
        else:                                             #else the gate must be I
            Pgate.append("I")
            
    Pgate = "".join(Pgate)
    
    if symp_gate[2][0]==1:                                #if the phase bit is 1 then the gate must be negative
        Pgate = "-" + Pgate
        
    return Pgate

In [ ]:
#here's all the base gates we will be using
I = np.array([[1,0],
              [0,1]])

X = np.array([[0,1],
              [1,0]])

Z = np.array([[1,0],
              [0,-1]])

Y = np.array([[0,-1j],
              [1j,0]])

In [ ]:
#heres a list of all the gates possible in pauli string format
gate_names = ["II", "IX", "IY", "IZ", "XI", "XX", "XY", "XZ", 
              "YI", "YX", "YY", "YZ", "ZI", "ZX", "ZY", "ZZ",
              "-II", "-IX", "-IY", "-IZ", "-XI", "-XX", "-XY", "-XZ", 
              "-YI", "-YX", "-YY", "-YZ", "-ZI", "-ZX", "-ZY", "-ZZ"]

#heres all the gates represented in 4x4 matrix form
II = np.kron(I,I)
IX = np.kron(I,X)
IY = np.kron(I,Y)
IZ = np.kron(I,Z)

XI = np.kron(X,I)
XX = np.kron(X,X)
XY = np.kron(X,Y)
XZ = np.kron(X,Z)

YI = np.kron(Y,I)
YX = np.kron(Y,X)
YY = np.kron(Y,Y)
YZ = np.kron(Y,Z)

ZI = np.kron(Z,I)
ZX = np.kron(Z,X)
ZY = np.kron(Z,Y)
ZZ = np.kron(Z,Z)

#heres a dictionary of all the gate names and matrix representations so I can call them easier
gate_dict = {"II":II, "IX":IX, "IY":IY, "IZ":IZ, "XI":XI, "XX":XX, "XY":XY, "XZ":XZ, 
              "YI":YI, "YX":YX, "YY":YY, "YZ":YZ, "ZI":ZI, "ZX":ZX, "ZY":ZY, "ZZ":ZZ,
              "-II":-II, "-IX":-IX, "-IY":-IY, "-IZ":-IZ, "-XI":-XI, "-XX":-XX, "-XY":-XY, "-XZ":-XZ, 
              "-YI":-YI, "-YX":-YX, "-YY":-YY, "-YZ":-YZ, "-ZI":-ZI, "-ZX":-ZX, "-ZY":-ZY, "-ZZ":-ZZ}

## Finding the combos
In order to find the 60 posible symplectic combinations we must outline the rules the combinations need to follow. To do this I will show how we come to find that there are 60 suitable combinations of symplectic basis.

### Symplectic overview
The Pauli representation is the gates' most typical form. An example of such representation is $X \otimes I \otimes Z \otimes Y$. This shows that an $X$ gate acts on the first qubit, the second qubit is unchanged, a $Z$ gate acts on the third qubit, and $Y$ acts on the fourth qubit. Usually when represented in this way the tensor product notation is collapsed on, such that the gates would be instead written as $XIZY$. Noting that $(X)(Z) = iY$, these gates may also be represented by the following equation:

$$\mathcal{P} = (-1)^c(-i)^{\vec{q}\cdot\vec{p}}(X)^\vec{q}(Z)^\vec{p}$$

The symplectic representation of quantum gates is another way of describing whether an X and/or Z gate is acting on a qubit. It uses the equation above and hilights just the vectors $\vec{q},\vec{p}$ and $c$. For instance the example gate above may be rewritten in its symplectic form as:

$$XIZY \rightarrow \left(--\vec{q}--|--\vec{p}--|c\right) = \left(1,0,0,1|0,0,1,1|0\right)$$

The advantage of this representation is multiplication of gates becomes addition of their symplectic binary strings:
$$\begin{align*}
(X ⊗ I ⊗ Z ⊗ Y )(−X ⊗ Y ⊗ X ⊗ X) &= [1, 0, 0, 1|0, 0, 1, 1|0]\\
&+ [1, 1, 1, 1|0, 1, 0, 0|1]\\
&= [0, 1, 1, 0|0, 1, 1, 1|1]\\
&= −I ⊗ Y ⊗ Y ⊗ Z
\end{align*}$$

It should be noted that the addition is only perfect if the twisted inner product of the two gates is equal to zero:

$$g_1 \odot g_2 = \vec{q}_1\cdot\vec{p}_2 \oplus \vec{q}_2\cdot\vec{p}_1 = 0$$

This ensures there are no unwanted imaginary phases, and hence why we may regard the phase bit as only being binary and not quaternary.

### The two qubit case, before deductions
For the two qubit case we have gates in the form $g = \left(x_1,x_2|z_1,z_2|r\right)$. For now I will ignore the phase, so we are left with $g = \left(x_1,x_2|z_1,z_2\right)$. This gives $2^{2n} = 2^4 = 16$ possible combinations for the gates. Since a symplectic basis for two qubits is represented using two gates;

$$
\begin{pmatrix} ---g_1--- \\ ---g_2---\end{pmatrix}
$$

we are left with $(2^4)(2^4) = 256$ posibilities.

### $g_1$ can't be Identity
Our first deduction will be the identity gate from $g_1$. This is because it is trivial, every state is stabilized by the identity, so we find nothing interesting by keeping it around. I shall keep the identity in $g_2$ for another little bit, but not long.

Possible bases: $(2^4-1)(2^4) = 240$

### Twisted inner product of the Symplectic form
The next limitation we need is that the twisted inner product of the gates' symplectic forms must equal zero mod 2;
$$g_1 \odot g_2 = 0 \;\mathbb{Z}_2$$

Every $g_2$ must follow this rule for every $g_1$. Since we are working with mod 2, and because $g_2$ iterates over every possile combination of four zeros and ones, it is safe to assume we will get rid of half of the $g_2$s for every $g_1$

Possible bases: $(2^4-1)(2^3) = 120$

### Linearity
The gates in the symplectic basis must be linearly independant of one another. This means for every $g_1$, we get rid of the identity (as $0\cdot g_1 = \mathbf{I}_4$) and its copy ($1\cdot g_1 = g_1$).

Possible bases: $(2^4-1)(2^3-2) = 90$

### Double counting
Given two gates $g_1$ and $g_2$, we can calculate a gate $g_3$ also in that basis via the product of the first two. This new gate can be swapped in for $g_2$, and we still have the same basis;
$\begin{pmatrix} ---g_1--- \\ ---g_2---\end{pmatrix}$ where $g_1 \cdot g_2 = g_3$ has the same basis as $\begin{pmatrix} ---g_1--- \\ ---g_3---\end{pmatrix}$ where $g_1 \cdot g_3 = g_2$ (still ignoring phases for now).This halves the amount of possible choices for $g_2$.

Possible bases: $(2^4-1)(2^2-1) = 45$

### Further Triple counting
Using the same concept as above, we can show that the following representations form the same basis:

$$\begin{pmatrix} ---g_1--- \\ ---g_2---\end{pmatrix} \; , \; \begin{pmatrix} ---g_2--- \\ ---g_3---\end{pmatrix} \;,\; \begin{pmatrix} ---g_3--- \\ ---g_1---\end{pmatrix}$$

This further divides our list of choices by 3, leaving us with a grand total number of symplectic basis of $\frac{1}{3}(2^4-1)(2^2-1) = 15$

### Bringing the Phases back
For every gate we have we can use a plus or minus form, ie we can use $[+g_1,+g_2], [+g_1,-g_2], [-g_1,+g_2], [-g_1,-g_2]$
This multiplies the number of basis by 4: $\frac{4}{3}(2^4-1)(2^2-1) = 60$

### Implementation
Now that I have outlined the steps we need to find the symplectic bases, I just need to implement them. I will get rid of the identity from $g_1$, check the twisted inner product, apply linearity and scrap any copies. 

In [ ]:
#this function calculates the twisted inner product of two gates
#expects string form of gate
def twisted_prod(g1,g2):
    g1 = symplectify(g1)
    g2 = symplectify(g2)
    prod = g1[0][0]*g2[1][0] + g1[0][1]*g2[1][1] + g1[1][0]*g2[0][0] + g1[1][1]*g2[0][1]
    return prod%2

In [ ]:
#this is an empty list we will add my combinations to
combos = []

#this loops over all the gate names, and firstly gets rid of all identities
#we know we probably could avoid complications if we removed the identities from the list of gate_names
#but we wish to be thorough
for g1 in range(len(gate_names)):
    if gate_names[g1] == "II":
        continue
    if gate_names[g1] == "-II":
        continue
        
    #next we loop through the gate_names list again for g_2, implementing the rules outlined above
    for g2 in range(len(gate_names)):
        
        #linearity
        if gate_names[g2] == "II":
            continue
        if gate_names[g2] == "-II":
            continue
        if gate_names[g1] == gate_names[g2]:
            continue
            
        #twisted inner product
        if twisted_prod(gate_names[g1],gate_names[g2]) == 1:
            continue
            
        #after getting past all those checks the combo is created
        combo = [gate_names[g1], gate_names[g2], Paulify(symp_mul(symplectify(gate_names[g1]), symplectify(gate_names[g2])))]
        
        #the combo is sorted in alphabetical order so they can be easily checked against one another
        combo.sort()
        
        #can still sometimes get identities, so an extra identity check
        #probably need to check for -1*g_1, but this works just as well
        if ("-II" or "II") in combo:
            continue
        
        #if the combo isnt in the list, its appended
        if combo not in combos:
            combos.append(combo)
combos

In [ ]:
len(combos)

I have found all 60 possible gate combinations for the symplectic basis. Now to find the states! Again we use the following formula:

$$\rho_{\left|\psi\right\rangle} = \left|\psi\right\rangle \left\langle\psi\right| \propto \mathbf{I} \pm g_1 \pm g_2 \pm g_1 \cdot g_2$$

Once we have the density matrix, it is just a matter of finding the eigenvector that corresponds to the non-zero eigenvalue of $\rho_{\left|\psi\right\rangle}$. This should be our stabilizer state corresponding to the $g_1, g_2, \text{&}\; g_1 \cdot g_2$ combination used to calculate $\rho_{\left|\psi\right\rangle}$ above.

I left it at that, and decided that once I have all my states in this form it would be faster to write them out by hand than to create another bit of code to print them in their Dirac notation.

In [ ]:
#this just numbers the states for our own sanity
i = 0

vect_list = []

#run over all the combos
for combo in combos:
    i+=1
    print("_________________")
    
    #print the combo
    print(i,")", combo[0], combo[1], combo[2])
    
    #calculate and print the matrix
    mat = II + gate_dict[combo[0]] + gate_dict[combo[1]] + gate_dict[combo[2]]
    print(mat)
    
    #calculate the e.vects and e.vals associated with the matrix
    w,v=eig(mat)
    
    #print(np.round(w,2))
    #print(np.round(v,2))
    #find the e.vect associated with the non-zero e.val and print it
    print(np.round([v[0][np.argmax(w)], v[1][np.argmax(w)], v[2][np.argmax(w)], v[3][np.argmax(w)]],2))
    vect_list.append(np.round([v[0][np.argmax(w)], v[1][np.argmax(w)], v[2][np.argmax(w)], v[3][np.argmax(w)]],2))
    print("")